#### Criando banco de dados

In [1]:
import sqlite3

conn = sqlite3.connect('stackoverflow.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS questions(
    id INTERGER PRIMARY KEY,
    title TEXT,
    tag TEXT,
    body TEXT 
)
''')

conn.commit()
print('Banco de dados criado com sucesso')

#### Função que adiciona os dados a base

In [25]:
def save_question(questions,tag):
    for question in questions:
        cursor.execute(''' INSERT OR REPLACE INTO questions (id,title,tag,body) VALUES (?,?,?,?) ''',
            (question['question_id'],question['title'],tag,question['body']))
    conn.commit()
    print(f'Tema {question} salvo na base')


#### Caputra as perguntas do stackoverflow

In [2]:
import httpx


url = "https://api.stackexchange.com/2.3/questions"
site = "stackoverflow"
api_key = "rl_sVA4bTddeGTtj2EFpzt2RKu7r"
page_size = 100
max_page = 5
assuntos = ['pandas','tensorflow','machine-learning','dataframe','scikit-learn',
            'data-visualization']

def get_stackoverflow_questions(tag, page):
    params = {
        "order": "desc",
        "sort": "activity",
        "tagged": tag,
        "site": site,
        "page": page,
        "filter" : "withbody",
        "page_size" : page_size,
        "key" : api_key,
        "is_answered" : 'True'
    }
    response = httpx.get(url, params=params)
    response.raise_for_status()  # Levanta um erro se a requisição falhar
    return response.json()

# Exemplo de uso

base = {
    'pandas' : [],
    'tensorflow': [],
    'machine-learning': [],
    'dataframe' : [],
    'scikit-learn' : [],
    'data-visualization' : []
}

for assunto in assuntos:
  for i in range(1,max_page+1):
      question = get_stackoverflow_questions(assunto, i)
      for item in question['items']:
        body ={
          'question_id' : item['question_id'],
          'title' : item['title'],
          'body' :  item['body']
        }
        base[assunto].append(body)


for tema in base:
    save_question(base[tema],tema)

KeyboardInterrupt: 

#### Trata o corpo do texto recebidos pela API

In [3]:
from bs4 import BeautifulSoup


def pross_text(text):
    soup = BeautifulSoup(text,'html.parser')

    #Queremos que a pergunta saia na mesma ordem mas com a distinção de texto e codigo 

    elements_order = [] #Criamos uma lista para amarzenar o texto na ordem do texto original

    #Itera pelos filhos direto do corpo do texto
    for elem in soup.find_all(['p','pre']):
        if elem.name == 'p':
            text = elem.get_text().strip()
            if text:
                elements_order.append(('texto',text))
        elif elem.name == 'pre':
            code = elem.get_text().strip()
            if code:
                elements_order.append(('codigo',code))

    texto_final = ''

    for tipo,conteudo in elements_order:
        if tipo == 'texto':
            texto_final += conteudo + '\n'
        elif tipo == 'codigo':
            texto_final += '```python \n' + conteudo + '\n'

    
    return texto_final

#### Atualiza a base de dados com os texto tratados

In [ ]:
cursor.execute('Select id,body from questions')
text = cursor.fetchall()

sql_update = "UPDATE questions SET body = ? WHERE id = ?"
for question in text:
    text = pross_text(question[1])
    cursor.execute(sql_update,(text,question[0]))
    conn.commit()

print('Tabela atualizada')
    

#### Agora vamos pegar as respostas no stackoverfow

In [32]:
cursor.execute("ALTER TABLE questions ADD COLUMN score INTERGER")
cursor.execute("ALTER TABLE questions ADD COLUMN answer TEXT")
conn.commit()

#### Adiciona a respostas no banco de dados

In [4]:
import httpx

cursor.execute('select id from questions')
ids = cursor.fetchall()
print(ids)
for id in ids:
    try:
        url = (f'https://api.stackexchange.com/2.3/questions/{id[0]}/answers?order=desc&sort=activity&site=stackoverflow&filter=withbody&key=rl_sVA4bTddeGTtj2EFpzt2RKu7r')
        response = httpx.get(url)
        response.raise_for_status()  # Levanta um erro se a requisição falhar
        try:
            score = response.json()['items'][0]['score']
            body = response.json()['items'][0]['body']
            trated_body = pross_text(body)
            sql_insert = "UPDATE questions SET score = ?,answer = ? WHERE id = ?"
            cursor.execute(sql_insert,(score,trated_body,id[0]))
        except:
            sql_insert = "UPDATE questions SET score = ?, anwser = ? WHERE id = ?"
            cursor.execute(sql_insert,(0,'no_answer',id[0]))
    except:
        print(f'Não foi encontrada resposta para {id[0]}')
conn.commit()

[(9917545,), (10327260,), (11040626,), (11311410,), (11707586,), (12127635,), (16381577,), (16958499,), (17381006,), (20444087,), (22155951,), (22231592,), (23539832,), (24788200,), (26147180,), (27023210,), (28805186,), (29576430,), (30368942,), (30502284,), (30984019,), (31107740,), (31247460,), (31483625,), (32428193,), (33459864,), (33984737,), (34524084,), (34674797,), (36573046,), (37452073,), (37571376,), (37679679,), (38223461,), (38426117,), (38733220,), (39331218,), (39525716,), (39662398,), (40142835,), (40705614,), (40780033,), (40845304,), (40880925,), (41472951,), (41523005,), (41641205,), (41666627,), (41769882,), (41859311,), (42462530,), (43029589,), (43237124,), (43382716,), (43556344,), (43640546,), (43992230,), (45548426,), (45968052,), (46113732,), (47325204,), (47434224,), (47509994,), (47633546,), (47639830,), (47717818,), (48035887,), (48079973,), (48087676,), (48200136,), (48252006,), (48510741,), (48758383,), (48856454,), (48891538,), (49819971,), (50957340,),

In [7]:
cursor.execute('select id from questions where answer is not NULL')
print(len(cursor.fetchall()))

554
